In [1]:
import gym
import ray
from gym.spaces import Discrete, Tuple
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
#from ray.rllib.models import ModelCatalog, ModelV2
from ray.rllib.models.tf.misc import normc_initializer
#from ray.rllib.models.utils import get_filter_config
from ray.rllib.utils import override
from scouting_gym.tasks.scouting_discrete_task import ScoutingDiscreteTask
import matplotlib.pyplot as plt
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term
None


In [2]:
env = gym.make('Scouting-v0')

print(env.observation_space)

[ERROR] [1611930464.665162, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1611930464.668641, 0.000000]: Start Init ControllersConnection
[WARN] [1611930464.669565, 0.000000]: END Init ControllersConnection


Tuple(Box(0.0, 4.0, (18,), float32), Box(-10.0, 10.0, (2,), float32), Box(-1.0, 1.0, (2,), float32))


In [3]:
obs = env.reset()
obs[2]

array([ 0.91796736, -1.        ])

In [4]:
obs = env.reset()
plt.figure()
thetas = np.linspace(-2.44, 2.44, 18)
thetas = np.linspace(-3.14, 3.14, 18)
plt.polar(thetas, obs[0])
plt.show()


-1.9829870348639111 -1.744520828712645


In [30]:
env.reset()
for _ in range(40):
    #obs, reward, done, _ = env.step(1)
    obs, reward, done, _ = env.step([0.6, ])
print(obs[0].min())
print(obs[0].max())

#plt.imshow(obs[0].squeeze())

[2.0461597  0.69780225] [0.00249521 0.00662212]
(array([1.8497185 , 0.9778511 , 0.76388377, 0.66989803, 0.9958483 ,
       1.1698219 , 1.3018017 , 1.4397807 , 2.9415522 , 2.081683  ,
       2.1636705 , 2.291651  , 2.6735928 , 2.317647  , 1.8277217 ,
       1.1878191 , 1.2298127 , 3.99      ], dtype=float32), array([2.0461597 , 0.69780225], dtype=float32), array([0.00249521, 0.00662212], dtype=float32))
[-0.2328857868381443, -1.1929179055514443, -0.11370100552919662]


(array([1.8857129 , 1.0198448 , 0.74788606, 0.6798965 , 1.1478251 ,
        1.1598234 , 1.293803  , 1.4057859 , 2.929554  , 2.0336905 ,
        2.1256764 , 2.265655  , 2.705588  , 2.3196468 , 1.8357207 ,
        1.2138152 , 1.2358118 , 3.99      ], dtype=float32),
 array([2.0328858, 0.6929179], dtype=float32),
 array([-0.99, -0.99], dtype=float32))

In [5]:
#print(obs)


(array([2.371639 , 1.5197686, 1.083835 , 1.5697609, 3.99     , 1.7457341,
       0.9118612, 0.781881 , 0.6758971, 0.6519007, 1.18182  , 1.333797 ,
       1.549764 , 3.1415217, 2.2296605, 2.3456428, 2.371639 , 2.547612 ],
      dtype=float32), array([0.0542413 , 0.32386878], dtype=float32), array([0.99     , 0.1522624], dtype=float32))


In [6]:
import tensorflow as tf
from ray.rllib.models.tf.tf_modelv2 import TFModelV2

class CNNPlusFCConcatModel(TFModelV2):
    """TFModelV2 concat'ing CNN outputs to flat input(s), followed by FC(s).

    Note: This model should be used for complex (Dict or Tuple) observation
    spaces that have one or more image components.
    """

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        # TODO: (sven) Support Dicts as well.
        assert isinstance(obs_space.original_space, (Tuple)), \
            "`obs_space.original_space` must be Tuple!"

        super().__init__(obs_space, action_space, num_outputs, model_config,
                         name)

        # Build the CNN(s) given obs_space's image components.
        self.cnns = {}
        concat_size = 0
        for i, component in enumerate(obs_space.original_space):
            # Image space.
            if len(component.shape) == 3:
                config = {
                    "conv_filters": [[16, [3, 3], 2], [32, [3, 3], 2], [64, [3, 3], 2], [128, [5, 16], 1]],
                    "conv_activation": model_config.get("conv_activation"),
                }
                cnn = ModelCatalog.get_model_v2(
                    component,
                    action_space,
                    num_outputs=None,
                    model_config=config,
                    framework="tf",
                    name="cnn_{}".format(i))
                concat_size += cnn.num_outputs
                self.cnns[i] = cnn
                self.register_variables(cnn.base_model.variables)
            # Discrete inputs -> One-hot encode.
            elif isinstance(component, Discrete):
                concat_size += component.n
            # TODO: (sven) Multidiscrete (see e.g. our auto-LSTM wrappers).
            # Everything else (1D Box).
            else:
                assert len(component.shape) == 1, \
                    "Only input Box 1D or 3D spaces allowed!"
                concat_size += component.shape[-1]

        self.logits_and_value_model = None
        self._value_out = None
        if num_outputs:
            # Action-distribution head.
            concat_layer = tf.keras.layers.Input((concat_size, ))
            logits_layer = tf.keras.layers.Dense(
                num_outputs,
                activation=tf.keras.activations.linear,
                name="logits")(concat_layer)

            # Create the value branch model.
            value_layer = tf.keras.layers.Dense(
                1,
                name="value_out",
                activation=None,
                kernel_initializer=normc_initializer(0.01))(concat_layer)
            self.logits_and_value_model = tf.keras.models.Model(
                concat_layer, [logits_layer, value_layer])
        else:
            self.num_outputs = concat_size
        self.register_variables(self.logits_and_value_model.variables)

    @override(ModelV2)
    def forward(self, input_dict, state, seq_lens):
        # Push image observations through our CNNs.
        outs = []
        for i, component in enumerate(input_dict["obs"]):
            if i in self.cnns:
                cnn_out, _ = self.cnns[i]({"obs": component})
                outs.append(cnn_out)
            else:
                outs.append(component)
        # Concat all outputs and the non-image inputs.
        out = tf.concat(outs, axis=1)
        if not self.logits_and_value_model:
            return out, []

        # Value branch.
        logits, values = self.logits_and_value_model(out)
        self._value_out = tf.reshape(values, [-1])
        return logits, []

    @override(ModelV2)
    def value_function(self):
        return self._value_out

ModelCatalog.register_custom_model("CNNPlusFCConcatModel", CNNPlusFCConcatModel)

NameError: name 'override' is not defined

In [3]:
config = {
    "env": ScoutingDiscreteTask,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": 1,
    "num_workers": 1,  # parallelism
    "model": {
        #"custom_model": "CNNPlusFCConcatModel",
        #"conv_filters": [[16, [4, 3], 2], [32, [3, 3], 2], [64, [3, 3], 2], [128, [5, 16], 1]],
        "fcnet_hiddens": [256, ],
        #"use_lstm": True,
        #"lstm_cell_size": 256,
        #"fcnet_hiddens": tune.grid_search([[64, 64, ], [128, 128, ], [256, 256, ]])
    }
}

stop = {
    "episodes_total": 4500,
}

In [4]:
ray.init()

2021-01-29 15:27:56,575	INFO resource_spec.py:223 -- Starting Ray with 14.11 GiB memory available for workers and up to 7.07 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2021-01-29 15:27:57,138	INFO services.py:1191 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.178.60',
 'raylet_ip_address': '192.168.178.60',
 'redis_address': '192.168.178.60:6379',
 'object_store_address': '/tmp/ray/session_2021-01-29_15-27-56_573766_4155763/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-29_15-27-56_573766_4155763/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2021-01-29_15-27-56_573766_4155763'}

In [5]:
def train(stop_criteria, config, restorepath):
    """
    Train an RLlib PPO agent using tune until any of the configured stopping criteria is met.
    :param stop_criteria: Dict with stopping criteria.
        See https://docs.ray.io/en/latest/tune/api_docs/execution.html#tune-run
    :return: Return the path to the saved agent (checkpoint) and tune's ExperimentAnalysis object
        See https://docs.ray.io/en/latest/tune/api_docs/analysis.html#experimentanalysis-tune-experimentanalysis
    """
    analysis = ray.tune.run(PPOTrainer, config=config,
                            stop=stop_criteria,
                            checkpoint_freq=1,
                            checkpoint_at_end=True, restore=restorepath)
    # list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value
    checkpoints = analysis.get_trial_checkpoints_paths(trial=analysis.get_best_trial('episode_reward_mean', mode='max'),
                                                       metric='episode_reward_mean',
                                                       )
    # retriev the checkpoint path; we only have a single checkpoint, so take the first one
    checkpoint_path = checkpoints[0][0]
    return checkpoint_path, analysis

def load(checkpoint_path, config):
    """
    Load a trained RLlib agent from the specified path. Call this before testing a trained agent.
    :param path: Path pointing to the agent's saved checkpoint (only used for RLlib agents)
    """
    agent = PPOTrainer(config=config)
    agent.restore(checkpoint_path)
    return agent

def test(agent, env):
    """Test trained agent for a single episode. Return the episode reward"""
    # instantiate env class

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward

    return episode_reward

In [7]:
checkpoint_path, analysis = train(stop_criteria=stop,
                                  config=config,
                                  restorepath='/home/dschori/ray_results/PPO_2021-01-28_15-51-43/PPO_ScoutingDiscreteTask_56467_00000_0_2021-01-28_15-51-43/checkpoint_375/checkpoint-375')


In [17]:
checkpoint_number = 375
checkpoint_path = '/home/dschori/ray_results/PPO_2021-01-28_15-51-43/' \
                  'PPO_ScoutingDiscreteTask_56467_00000_0_2021-01-28_15-51-43/' \
                  'checkpoint_{}/checkpoint-{}'.format(checkpoint_number, checkpoint_number)
agent = load(checkpoint_path=checkpoint_path, config=config)

[ERROR] [1611930847.749545, 57.967000]: NOT Initialising Simulation Physics Parameters
[WARN] [1611930847.752978, 57.967000]: Start Init ControllersConnection
[WARN] [1611930847.753989, 57.967000]: END Init ControllersConnection
(pid=4155868) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=4155868) Instructions for updating:
(pid=4155868) non-resource variables are not supported in the long term
2021-01-29 15:34:11,106	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2021-01-29 15:34:11,200	INFO trainable.py:472 -- Restored on 192.168.178.60 from checkpoint: /home/dschori/ray_results/PPO_2021-01-28_15-51-43/PPO_ScoutingDiscreteTask_56467_00000_0_2021-01-28_15-51-43/checkpoint_375/checkpoint-375
2021-01-29 15:34:11,201	INFO trainable.py:480 -- Current state after restoring: {'_

(pid=4155868) None


In [18]:
import time
time_now = time.time()
while True:
    if time.time() - time_now > 80.:
        break
    episode_reward = test(agent=agent, env=env)